**Transition States**

After we have defined some stable minima, we often need to work out the energy to travel between these states. Let's deminstrate this using our Pt110 slab again.

Our first step in these types of calculations is again to setup our environment and building the Pt110 model, this time with a Cu atom on the surface in a "start" and "finish" position:

In [1]:
from ase import Atoms, Atom
from ase.build import fcc110
from math import sqrt

# Make the Pt 110 slab.
initial = fcc110('Pt', (2, 2, 4), vacuum=7.)
initial.set_pbc((1, 1, 0))

a = 3.9242
b = a / sqrt(2)
h = b / 2 
initial.append(Atom('Cu',  initial[8].position + (0., 0., 2.5)))
final = initial.copy()
final.positions[-1, 1] += b

We can look at these structures if we want in the normal way:

In [2]:
from ase.visualize import view
view([initial, final], viewer='ngl')

To find the transition state, we must interpolate between these "start" and "final" structures, which we have some tools for; we are also at the same time going to setup constraints and allocate the appropriate `EMT` calculator:

In [3]:
# Construct a list of images:
images = [initial]
for i in range(5):
    images.append(initial.copy())
images.append(final)

from ase.constraints import FixAtoms
# Make a mask of zeros and ones that select fixed atoms (the
# two bottom layers):
mask = initial.positions[:, 2] - min(initial.positions[:, 2]) < 3.5 * h
constraint = FixAtoms(mask=mask)

from ase.calculators.emt import EMT
for image in images:
    # Let all images use an EMT calculator:
    image.set_calculator(EMT())
    image.set_constraint(constraint)

Now, to make sure we get a "true" transition state we need to optimise the starting and finishing structures:

In [4]:
from ase.optimize import BFGS
# Relax the initial and final states:
BFGS(initial).run(fmax=0.05)
BFGS(final).run(fmax=0.05)

      Step     Time          Energy         fmax
BFGS:    0 08:32:46        5.560884        0.1971
BFGS:    1 08:32:46        5.560360        0.1756
BFGS:    2 08:32:46        5.558414        0.0086
      Step     Time          Energy         fmax
BFGS:    0 08:32:46        5.560891        0.1971
BFGS:    1 08:32:46        5.560366        0.1756
BFGS:    2 08:32:46        5.558418        0.0094


True

With these optimised structures, we can now build and interpolate a transition state search with the "nudged elastic band approach":

In [5]:
from ase.neb import NEB
# Create a Nudged Elastic Band:
neb = NEB(images)

# Make a starting guess for the minimum energy path (a straight line
# from the initial to the final state):
neb.interpolate()

from ase.optimize import MDMin
# Relax the NEB path:
minimizer = MDMin(neb)
minimizer.run(fmax=0.05)

       Step     Time          Energy         fmax
MDMin:    0 08:32:46        6.376949        2.1160
MDMin:    1 08:32:46        6.294347        1.7889
MDMin:    2 08:32:47        6.139623        0.9546
MDMin:    3 08:32:47        6.071327        0.0545
MDMin:    4 08:32:47        6.073959        0.5514
MDMin:    5 08:32:47        6.073399        0.4876
MDMin:    6 08:32:48        6.072183        0.3014
MDMin:    7 08:32:48        6.071332        0.0344


True

Note how the force (`fmax`) decreases as the calculation proceeds, as does the energy.

Finally, as in other scenarios it is always good to check the results!

In [6]:
#Print the relative energetics of the models on the pathway:
for atoms in images:
    print("Energy: ", atoms.get_potential_energy())

#Bring up the viewer for the atomic objects
view(images, viewer='ngl')

Energy:  5.558413829913128
Energy:  5.7058225489332095
Energy:  5.965744473836438
Energy:  6.071332302410243
Energy:  5.964642539299511
Energy:  5.70452134793141
Energy:  5.558418144324813


If you want to push yourself further, have a play with removing the constraints and see what affect that has on the overal energetics? Does it decrease or increase the barrier, and why?

(_Hint_: To do this you want to remove the constraints above and rerun all the subsequent boxes!)

**Vibrations**

If we want to confirm the nature of a transition state, we need to calculate the vibrational frequencies and ensure we have one imaginary mode!

Let's calculate the vibrations for the start, finish and transition structure so we can compare:

In [7]:
from ase.vibrations import Vibrations
vib = Vibrations(initial)
vib.run()
vib.summary()
# Save the highest energy vibrational mode to visualise
vib.write_mode(-1)

---------------------
  #    meV     cm^-1
---------------------
  0    8.0i     64.4i
  1    0.0i      0.0i
  2    0.0i      0.0i
  3    0.0i      0.0i
  4    0.1       1.2 
  5    2.5      20.5 
  6    3.8      30.7 
  7    4.0      32.4 
  8    4.6      37.3 
  9    4.8      38.9 
 10    5.3      43.1 
 11    5.5      44.5 
 12    5.7      46.2 
 13    6.0      48.0 
 14    6.6      53.4 
 15    6.7      53.8 
 16    6.8      55.1 
 17    6.9      55.7 
 18    7.2      58.2 
 19    7.2      58.3 
 20    7.5      60.8 
 21    7.6      61.6 
 22    7.7      61.8 
 23    7.7      62.5 
 24    7.8      62.6 
 25    8.2      66.1 
 26    8.3      67.3 
 27    8.6      69.3 
 28    8.8      71.2 
 29    9.4      76.2 
 30    9.6      77.1 
 31   10.1      81.7 
 32   10.3      83.0 
 33   10.5      84.6 
 34   10.5      84.8 
 35   10.7      86.2 
 36   11.3      90.8 
 37   11.6      93.7 
 38   12.3      99.4 
 39   12.4     100.4 
 40   12.7     102.6 
 41   14.4     116.0 
 42   14.8 

Let's look at the strongest vibration:

In [8]:
from ase.io.trajectory import Trajectory
traj = Trajectory('vib.50.traj')
view(traj, viewer='ngl')

/Users/andrewlogsdail/Library/Python/3.8/lib/python/site-packages/ase/io/jsonio.py:122: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  a = np.array(obj)


We'll repeat the vibrations analysis for the final structure, just to compare briefly: 

In [9]:
# Delete old files
import os
files = os.listdir(os.curdir)
for f in files:
    if f.endswith(".pckl"):
        os.remove(f)

# Run new vibrations
from ase.vibrations import Vibrations
vib = Vibrations(final)
vib.run()
vib.summary()

Writing vib.eq.pckl
Writing vib.0x-.pckl
Writing vib.0x+.pckl
Writing vib.0y-.pckl
Writing vib.0y+.pckl
Writing vib.0z-.pckl
Writing vib.0z+.pckl
Writing vib.1x-.pckl
Writing vib.1x+.pckl
Writing vib.1y-.pckl
Writing vib.1y+.pckl
Writing vib.1z-.pckl
Writing vib.1z+.pckl
Writing vib.2x-.pckl
Writing vib.2x+.pckl
Writing vib.2y-.pckl
Writing vib.2y+.pckl
Writing vib.2z-.pckl
Writing vib.2z+.pckl
Writing vib.3x-.pckl
Writing vib.3x+.pckl
Writing vib.3y-.pckl
Writing vib.3y+.pckl
Writing vib.3z-.pckl
Writing vib.3z+.pckl
Writing vib.4x-.pckl
Writing vib.4x+.pckl
Writing vib.4y-.pckl
Writing vib.4y+.pckl
Writing vib.4z-.pckl
Writing vib.4z+.pckl
Writing vib.5x-.pckl
Writing vib.5x+.pckl
Writing vib.5y-.pckl
Writing vib.5y+.pckl
Writing vib.5z-.pckl
Writing vib.5z+.pckl
Writing vib.6x-.pckl
Writing vib.6x+.pckl
Writing vib.6y-.pckl
Writing vib.6y+.pckl
Writing vib.6z-.pckl
Writing vib.6z+.pckl
Writing vib.7x-.pckl
Writing vib.7x+.pckl
Writing vib.7y-.pckl
Writing vib.7y+.pckl
Writing vib.7z

As the structures are symmetry equivalent, we should have the same energetics for these two systems.

Finally, now lets consider the transition state:

In [10]:
# Delete old files
import os
files = os.listdir(os.curdir)
for f in files:
    if f.endswith(".pckl"):
        os.remove(f)

from ase.vibrations import Vibrations
vib = Vibrations(images[3])
vib.run()
vib.summary()
# Save the imaginary vibrational mode to visualise
vib.write_mode(0)

Writing vib.eq.pckl
Writing vib.0x-.pckl
Writing vib.0x+.pckl
Writing vib.0y-.pckl
Writing vib.0y+.pckl
Writing vib.0z-.pckl
Writing vib.0z+.pckl
Writing vib.1x-.pckl
Writing vib.1x+.pckl
Writing vib.1y-.pckl
Writing vib.1y+.pckl
Writing vib.1z-.pckl
Writing vib.1z+.pckl
Writing vib.2x-.pckl
Writing vib.2x+.pckl
Writing vib.2y-.pckl
Writing vib.2y+.pckl
Writing vib.2z-.pckl
Writing vib.2z+.pckl
Writing vib.3x-.pckl
Writing vib.3x+.pckl
Writing vib.3y-.pckl
Writing vib.3y+.pckl
Writing vib.3z-.pckl
Writing vib.3z+.pckl
Writing vib.4x-.pckl
Writing vib.4x+.pckl
Writing vib.4y-.pckl
Writing vib.4y+.pckl
Writing vib.4z-.pckl
Writing vib.4z+.pckl
Writing vib.5x-.pckl
Writing vib.5x+.pckl
Writing vib.5y-.pckl
Writing vib.5y+.pckl
Writing vib.5z-.pckl
Writing vib.5z+.pckl
Writing vib.6x-.pckl
Writing vib.6x+.pckl
Writing vib.6y-.pckl
Writing vib.6y+.pckl
Writing vib.6z-.pckl
Writing vib.6z+.pckl
Writing vib.7x-.pckl
Writing vib.7x+.pckl
Writing vib.7y-.pckl
Writing vib.7y+.pckl
Writing vib.7z

Note the single imaginary frequency! So we have a valid transition state. Let's also look at the highest energy vibration:

In [11]:
from ase.io.trajectory import Trajectory
traj = Trajectory('vib.0.traj')
view(traj, viewer='ngl')